<a href="https://colab.research.google.com/github/Anudeep27-bit/ETL-migration-workflow-Alteryx-to-BigQuery-/blob/main/Final_NA_Billing_Debits%26_credits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.cloud import bigquery
client = bigquery.Client(project='irm-fin-acct-dp-prod')


In [2]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
query = """
Create or Replace TABLE irm-fin-acct-dp-prod.Etl_alt_BQ.billing_detail_snap AS(
SELECT billing_system,  invoicing_country_code, calendar_date,  invoice_date, invoice_id, year_month, transaction_country_name, transaction_cluster,  transaction_region, customer_L1_code, customer_L1_name, imga, billing_cycle,  global_industry_indicator,  sf_country_parent_id, sf_country_parent_name, oracle_account_number,  oracle_account_name,  salesforce_billing_account, bill_code,  description,  product_id, product_summary_type, bill_code_data_source,  gl_account, gl_account_description, revenue_classification, currency, rate, SUM(quantity) as quantity, number_of_months,  SUM(amount) as amount, rate_USD, amount_USD, bill_code_sfdc, unit_of_measure,  storage_from_date,  storage_to_date,  original_invoice_id,  revenue_recognition_date, bill_in_advance_flag, bill_in_advance_month_number, revenue_recognition_amount, allocated_quantity
 FROM `irm-fin-acct-dp-prod.rpt_billing.billing_detail` where (transaction_region = "NA") and transaction_country_name != "Canada" AND calendar_date >= "2023-06-01" and calendar_date <= "2024-08-01" and revenue_classification ="Storage" and product_summary_type = "Records Management"
Group by ALL)
"""
client.query(query).result()
print(" table created successfully.")


In [4]:
query = """
CREATE OR REPLACE TABLE `irm-fin-acct-dp-prod.Etl_alt_BQ.billing_final_output_table` AS
WITH
-- First filtering step
cte_1 AS (
  SELECT *
  FROM `irm-fin-acct-dp-prod.Etl_alt_BQ.billing_detail_snap`
  WHERE description NOT IN ('STORAGE, DESTROYED', 'STORAGE, WITHDRAWN')
    AND description NOT LIKE '%PW\\'D%'
    AND description NOT LIKE '%DEST%'
),

-- Second filtering: filter out rows where amount != 0
cte_2 AS (
  SELECT *
  FROM cte_1
  WHERE amount != 0
),

-- Country transformation
cte_3 AS (
  SELECT *,
         CASE
           WHEN transaction_country_name = 'US' THEN 'United States'
           ELSE transaction_country_name
         END AS Country
  FROM cte_2
),

-- Create table for negative amounts with Cycle = 'credits'
negative_amounts AS (
  SELECT *, 'credits' AS Cycle
  FROM cte_3
  WHERE amount < 0
),

-- Create table for non-negative amounts
non_negative_amounts AS (
  SELECT *
  FROM cte_3
  WHERE amount >= 0
),

-- Summarize negative amounts
cte_4 AS (
  SELECT
    `Year_Month`,
    `customer_L1_code`,
    -amount AS amount,  -- Negate the amount values
    COUNT(*) AS Count
  FROM negative_amounts
  GROUP BY `Year_Month`, `customer_L1_code`, amount
),

-- Inner join of negative and non-negative amounts
joined_inner_amounts AS (
  SELECT
    non_neg.`Year_Month`,
    non_neg.`customer_L1_code`,
    non_neg.`amount` AS non_negative_amount,
    neg.`amount` AS negative_amount,
    'debits' AS Cycle  -- Add "Cycle" column with "debits"
  FROM non_negative_amounts AS non_neg
  JOIN negative_amounts AS neg
  ON non_neg.`Year_Month` = neg.`Year_Month`
  AND non_neg.`customer_L1_code` = neg.`customer_L1_code`
  AND non_neg.`amount` = neg.`amount`
),

-- Merged table (full outer join between joined_inner_amounts and negative_amounts)
merged_table AS (
  SELECT
    COALESCE(t1.Cycle, t2.Cycle) AS Cycle,
    COALESCE(t1.customer_L1_code, t2.customer_L1_code) AS customer_L1_code,
    t1.Year_Month,
    t1.negative_amount,
    t1.non_negative_amount,
    t2.bill_in_advance_flag,
    t2.currency,
    t2.oracle_account_number,
    t2.transaction_region,
    t2.Country,
    t2.transaction_cluster,
    t2.bill_in_advance_month_number,
    t2.oracle_account_name,
    t2.sf_country_parent_id,
    t2.storage_to_date,
    t2.allocated_quantity,
    t2.global_industry_indicator,
    t2.imga,
    t2.quantity,
    t2.invoice_date,
    t2.revenue_classification,
    t2.product_summary_type,
    t2.bill_code,
    t2.amount_USD,
    t2.number_of_months,
    t2.bill_code_sfdc,
    t2.revenue_recognition_date,
    t2.rate_USD,
    t2.customer_L1_name,
    t2.invoicing_country_code,
    t2.sf_country_parent_name,
    t2.invoice_id,
    t2.description,
    t2.gl_account,
    t2.billing_system,
    t2.bill_code_data_source,
    t2.unit_of_measure,
    t2.original_invoice_id,
    t2.year_month AS year_month_t2,  -- Renamed year_month to avoid conflict
    t2.revenue_recognition_amount,
    t2.transaction_country_name,
    t2.billing_cycle,
    t2.gl_account_description,
    t2.amount AS amount_t2,  -- Renamed amount to avoid conflict
    t2.rate,
    t2.storage_from_date,
    t2.product_id,
    t2.salesforce_billing_account,
    t2.calendar_date
  FROM joined_inner_amounts AS t1
  FULL OUTER JOIN negative_amounts AS t2
  ON t1.Cycle = t2.Cycle
  AND t1.customer_L1_code = t2.customer_L1_code
),

-- Final summarized table
final_query AS (
  SELECT
    Cycle,  -- Group by Cycle
    calendar_date AS `Transaction Date`,  -- Group by calendar_date, alias it as 'Transaction Date'
    Country,  -- Group by Country
    customer_L1_code AS CustID,  -- Group by customer_L1_code, alias it as CustID
    oracle_account_number AS `Oracle ID`,  -- Group by oracle_account_number, alias it as 'Oracle ID'
    ANY_VALUE(oracle_account_name) AS `Customer Name`,  -- Use ANY_VALUE to select a value without grouping
    SUM(amount_t2) AS Amount  -- Sum the amount_t2 column
  FROM merged_table
  GROUP BY
    Cycle, calendar_date, Country, customer_L1_code, oracle_account_number
)

-- Select final output
SELECT * FROM final_query;
"""

client.query(query).result()
print("Final output table created successfully.")


Final output table created successfully.
